## 0. Libarary 불러오기 및 경로설정

In [43]:
import os
import pandas as pd
from PIL import Image

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader

from torchvision import transforms
from torchvision.transforms import Resize, ToTensor, Normalize
import timm
import numpy as np
import albumentations as A
from albumentations.pytorch import ToTensorV2
import PIL

In [2]:
# 테스트 데이터셋 폴더 경로를 지정해주세요.
test_dir = '/opt/ml/input/data/eval'

## 1. Model 정의

In [3]:
class ENN_Model(nn.Module):
    def __init__(self, model_name, num_classes):
        super(ENN_Model, self).__init__()
        self.num_classes = num_classes
        self.model = timm.create_model(model_name, pretrained=True)

        n_features = self.model.classifier.in_features
        self.model.classifier = torch.nn.Linear(in_features=n_features, out_features=num_classes, bias=True)
        torch.nn.init.xavier_uniform_(self.model.classifier.weight)
        stdv = 1/np.sqrt(self.num_classes)
        self.model.classifier.bias.data.uniform_(-stdv, stdv)

    def forward(self, x):
        return self.model(x)

In [53]:
A_transform = {
    'train':
        A.Compose([
            A.Resize(512, 512),
            A.RandomCrop(384, 384),
            A.HorizontalFlip(p=0.5),
            A.Cutout(num_holes=8, max_h_size=32,max_w_size=32),
            A.ElasticTransform(),
            A.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
            ToTensorV2()
        ]),
    'valid':
        A.Compose([
            A.Resize(384, 384),
            A.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
            ToTensorV2()
        ]),
    'VIT_test':
        A.Compose([
            A.Resize(384, 384),
            A.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
            ToTensorV2()
        ]),
    'ENN_test':
        A.Compose([
            A.Resize(384, 384),
            A.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
            ToTensorV2()
        ])
}

# 이 위치에서 모델을 불러오세요

## 2. Test Dataset 정의

In [48]:
class TestDataset(Dataset):
    def __init__(self, img_paths, transform):
        self.img_paths = img_paths
        self.transform = transform

    def __getitem__(self, index):
        image = PIL.Image.open(self.img_paths[index])
        image = np.array(image.convert("RGB"))
        if self.transform:
            image = self.transform(image=image)
            image = image['image']
        return image

    def __len__(self):
        return len(self.img_paths)

## 3. Inference

In [49]:
from tqdm import tqdm

In [57]:
submission = pd.read_csv(os.path.join(test_dir, 'info.csv'))
image_dir = os.path.join(test_dir, 'new_images')

image_paths = [os.path.join(image_dir, img_id) for img_id in submission.ImageID]
dataset = TestDataset(image_paths, A_transform['VIT_test'])
test_loader = DataLoader(dataset, batch_size=1, shuffle=False, num_workers=8)

model.eval()
all_predictions = []
with tqdm(test_loader, total=test_loader.__len__(), unit="batch") as test_bar:
    for images in test_bar:
        with torch.no_grad():
            images = images.to(device)
            pred = model(images)
            pred = pred.argmax(dim=-1)
            all_predictions.extend(pred.cpu().numpy())
    
submission['ans'] = all_predictions
submission.to_csv(os.path.join(test_dir, 'submission_vit_0901_064031_14.csv'), index=False)
print('test inference is done!')

 20%|█▉        | 2468/12600 [01:16<05:13, 32.32batch/s]


KeyboardInterrupt: 

In [61]:
error_set = set([1, 2, 4, 5, 7, 8, 10, 11, 16, 17])

# 오분류 가능성 있는 것들

In [136]:
submission = pd.read_csv('/opt/ml/input/data/eval/submission_ml_image-classification-level1-04_saved_models_PretrainModelTimm_ViT_large_0826_155755_checkpoint-epoch9.pth.csv')

In [137]:
df_tocheck = submission[submission['ans'].apply(lambda x : x in error_set)]

In [138]:
class TestDataset(Dataset):
    def __init__(self, img_paths, transform):
        self.img_paths = img_paths
        self.transform = transform

    def __getitem__(self, index):
        image = PIL.Image.open(self.img_paths[index])
        image = np.array(image.convert("RGB"))
        if self.transform:
            image = self.transform(image=image)
            image = image['image']
        return image

    def __len__(self):
        return len(self.img_paths)

In [139]:
df_tocheck

,ImageID,ans
1,0e72482bf56b3581c081f7da2a6180b8792c7089.jpg,1
6,1903db7dcc1262d0441677afa422e6a8371e5b37.jpg,8
7,441419a874f4d031cd576850b68539ca7d35bedf.jpg,4
8,388856cd1ef99b1918273a827a75f2aff2478321.jpg,4
15,69ad31adcc3e6e8de7af886e970233fa61699045.jpg,4
...,...,...
12592,b8baa9fb4d8da1dd529c857fc74d91c1abacd19c.jpg,1
12595,d71d4570505d6af8f777690e63edfa8d85ea4476.jpg,1
12596,6cf1300e8e218716728d5820c0bab553306c2cfd.jpg,4
12598,030d439efe6fb5a7bafda45a393fc19f2bf57f54.jpg,1


In [140]:
check_image_paths = [os.path.join(image_dir, img_id) for img_id in df_tocheck.ImageID]

In [141]:
dataset_tocheck = TestDataset(check_image_paths, A_transform['ENN_test'])

dataset_tocheck_loader = DataLoader(
    dataset_tocheck,
    shuffle=False,
    batch_size = 118,
    num_workers=1,
    pin_memory=True,
)

In [142]:
model = ENN_Model('efficientnet_b3', 8).to('cuda')
model.load_state_dict(torch.load('./saved/models/20210902_085307_efficientnet_b3/baseline_efficientnet_b3_lr0.0005_batch24_kfold0_epoch17_valid_loss_0.64224.pt'))

<All keys matched successfully>

In [143]:
# 모델이 테스트 데이터셋을 예측하고 결과를 저장합니다. 
tocheck_predictions = []
for images in tqdm(dataset_tocheck_loader):
    with torch.no_grad():
        images = images.to(device)
        pred = model(images)
        pred = pred.argmax(dim=-1)
        tocheck_predictions.extend(pred.cpu().numpy())
df_tocheck['ans2'] = tocheck_predictions

100%|██████████| 56/56 [00:42<00:00,  1.33it/s]
<ipython-input-143-daecc33f65dc>:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_tocheck['ans2'] = tocheck_predictions


In [144]:
# 레이블 변환 함수
def trans_label(ans, ans2):
    # ans 원래 Label 18가지
    # ans 마스크 착용 여부만
    # ans2에서는 성별과 나이
    label = 0
    # 남성 30이상
    if ans2 <= 2:
        label = 1
    elif ans2 == 3:
        label = 2
    elif ans2 >= 4 and ans2 < 7:
        label = 4
    else:
        label = 5
    
    if ans >= 6 and ans <= 11:
        label += 6
    elif ans >= 12:
        label += 12
    return label
    

In [145]:
list(map(lambda x : 'm' if (x // 3) % 2 == 0 else 'f', list(range(18))))

['m',
 'm',
 'm',
 'f',
 'f',
 'f',
 'm',
 'm',
 'm',
 'f',
 'f',
 'f',
 'm',
 'm',
 'm',
 'f',
 'f',
 'f']

In [146]:
# 레이블 변환 함수
def trans_label(ans, ans2):
    # ans 원래 Label 18가지
    # ans 마스크 착용 여부만
    # ans2에서는 성별과 나이
    
    
    label = 0
    
    # Mask착용 여부
    if ans >= 6 and ans <= 11:
        label += 6
    elif ans >= 12:
        label += 12

    # 성별 여부
    if (ans // 3) % 2 == 1:
        label += 3

    # ans2에서 성별 제외
    if ans2 >= 4:
        ans2 -= 4
    # 나이에 대한 보정
    if ans2 <= 2:
        label += 1
    elif ans2 == 3:
        label += 2
    return label
        
        

In [147]:
for i in range(0, 8):
    print(trans_label(16, i))

16
16
16
17
16
16
16
17


In [148]:
trans_labels = []
for i in zip(df_tocheck.ans, df_tocheck.ans2):
    trans_labels.append(trans_label(*i))

In [149]:
import numpy as np

In [150]:
df_tocheck['ans_trans'] = np.array(trans_labels).reshape(-1, 1)

<ipython-input-150-4f3277734478>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_tocheck['ans_trans'] = np.array(trans_labels).reshape(-1, 1)


In [151]:
submission.loc[df_tocheck.index]['ans'] = df_tocheck.ans_trans

In [152]:
submission

,ImageID,ans
0,cbc5c6e168e63498590db46022617123f1fe1268.jpg,13
1,0e72482bf56b3581c081f7da2a6180b8792c7089.jpg,1
2,b549040c49190cedc41327748aeb197c1670f14d.jpg,13
3,4f9cb2a045c6d5b9e50ad3459ea7b791eb6e18bc.jpg,13
4,248428d9a4a5b6229a7081c32851b90cb8d38d0c.jpg,12
...,...,...
12595,d71d4570505d6af8f777690e63edfa8d85ea4476.jpg,1
12596,6cf1300e8e218716728d5820c0bab553306c2cfd.jpg,4
12597,8140edbba31c3a824e817e6d5fb95343199e2387.jpg,9
12598,030d439efe6fb5a7bafda45a393fc19f2bf57f54.jpg,1


In [153]:
# 제출할 파일을 저장합니다.
submission.to_csv(os.path.join(test_dir, 'submission.csv'), index=False)
print('test inference is done!')

test inference is done!
